In [ ]:
###Multi User support

# ===================== Imports =====================
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_core.runnables import Runnable
from langchain_core.messages import BaseMessage
from typing import TypedDict, Annotated, List, Dict
import os, operator, re, threading, queue, json
import pyttsx3
import sounddevice as sd
from vosk import Model, KaldiRecognizer
from dotenv import load_dotenv

# ===================== Setup =====================
# Load env vars
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

# Tools
arxiv_api = ArxivAPIWrapper(top_k_results=2, doc_content_chars_max=500)
arxiv = ArxivQueryRun(api_wrapper=arxiv_api, description="Query arxiv papers")

wiki_api = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper=wiki_api)

tavily = TavilySearchResults()
tools = [arxiv, wiki, tavily]

# LLM & Prompt
llm = ChatGroq(model="qwen-qwq-32b")
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI tutor. Only call tools like Wikipedia or Arxiv if needed."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])
output_parser = StrOutputParser()

# Agent State
class AgentState(TypedDict):
    input: str
    chat_history: Annotated[List[BaseMessage], operator.add]
    agent_scratchpad: list

# ===================== Multiuser Support =====================
class UserManager:
    def __init__(self):
        self.users: Dict[str, Dict] = {}
        self.current_user = None

    def add_user(self, user_id: str, user_name: str = None):
        """Add a new user with their own conversation memory"""
        if user_id not in self.users:
            memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
            agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)
            agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)

            self.users[user_id] = {
                "name": user_name or user_id,
                "memory": memory,
                "agent_executor": agent_executor,
                "conversation_count": 0
            }
            print(f"New user '{user_name or user_id}' added!")
        return self.users[user_id]

    def get_user(self, user_id: str):
        """Get user data, create if doesn't exist"""
        if user_id not in self.users:
            return self.add_user(user_id)
        return self.users[user_id]

    def set_current_user(self, user_id: str):
        """Set the current active user"""
        self.current_user = user_id
        user_data = self.get_user(user_id)
        print(f"👤 Switched to user: {user_data['name']}")

    def get_current_user(self):
        """Get current user data"""
        if not self.current_user:
            return None
        return self.users.get(self.current_user)

    def list_users(self):
        """List all users"""
        if not self.users:
            print("📝 No users registered yet.")
            return

        print("👥 Registered Users:")
        for user_id, user_data in self.users.items():
            status = " (current)" if user_id == self.current_user else ""
            print(f"  • {user_data['name']} - {user_data['conversation_count']} conversations{status}")

    def remove_user(self, user_id: str):
        """Remove a user and their conversation history"""
        if user_id in self.users:
            user_name = self.users[user_id]['name']
            del self.users[user_id]
            if self.current_user == user_id:
                self.current_user = None
            print(f"🗑️ User '{user_name}' removed.")
        else:
            print(f"❌ User '{user_id}' not found.")

# Initialize user manager
user_manager = UserManager()

# ===================== Voice Input (Vosk) =====================
model_path = r"C:\Users\DELL\Documents\AI CHATBOT\models\vosk-model-small-en-us-0.15\vosk-model-small-en-us-0.15"
model = Model(model_path)
recognizer = KaldiRecognizer(model, 16000)
audio_queue = queue.Queue()

def audio_callback(indata, frames, time, status):
    if status:
        print("Status:", status)
    audio_queue.put(bytes(indata))

def get_voice_input():
    print("🎤 Speak now (or type)...")
    with sd.RawInputStream(samplerate=16000, blocksize=8000, dtype='int16',
                           channels=1, callback=audio_callback):
        while True:
            data = audio_queue.get()
            if recognizer.AcceptWaveform(data):
                result = json.loads(recognizer.Result())
                text = result.get("text", "")
                if text:
                    print(" You said:", text)
                    return text

# ===================== Text Cleanup =====================
def clean_text(text):
    text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)
    text = re.sub(r'\*(.*?)\*', r'\1', text)
    text = re.sub(r'`(.*?)`', r'\1', text)
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def truncate_sentences(text, max_sentences=2):
    sentences = re.split(r'(?<=[.!?]) +', text)
    return ' '.join(sentences[:max_sentences])

# ===================== TTS (pyttsx3) =====================
def speak_output(text):
    def _speak():
        engine = pyttsx3.init()
        engine.setProperty("rate", 170)
        engine.setProperty("volume", 1.0)
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak, daemon=True).start()

# ===================== LangGraph Setup =====================
class CustomAgent(Runnable):
    def __init__(self, agent):
        self.agent = agent

    def invoke(self, state, config=None):
        response = self.agent.invoke(state, config)
        return {"chat_history": [response]}

def get_tools(state):
    messages = state.get("chat_history", [])
    tool_calls = [msg.tool_call for msg in messages if hasattr(msg, "tool_call")]
    return [tc["name"] for tc in tool_calls] if tool_calls else ["_end_"]

# ===================== Main Runner =====================
if __name__ == "__main__":
    print("🎙 Voice Tutor AI — Multiuser Edition")
    print("Commands: 'users', 'switch <user>', 'add <user>', 'remove <user>', 'exit'")
    print("Type 'exit' or 'quit' to stop.\n")

    # Initialize first user
    user_manager.set_current_user("default")

    while True:
        # Check if we have a current user
        current_user_data = user_manager.get_current_user()
        if not current_user_data:
            print("❌ No active user. Please add a user first.")
            user_input = input("Enter user ID: ").strip()
            if user_input.lower() in ['exit', 'quit']:
                break
            user_manager.set_current_user(user_input)
            continue

        current_user_name = current_user_data['name']
        mode = input(f"👤 {current_user_name} - Press [v] for voice, [t] to type, or enter command: ").strip().lower()

        if mode == "exit" or mode == "quit":
            print("👋 Exiting.")
            break

        # Handle user management commands
        if mode.startswith("switch "):
            user_id = mode[7:].strip()
            user_manager.set_current_user(user_id)
            continue
        elif mode.startswith("add "):
            user_id = mode[4:].strip()
            user_manager.add_user(user_id)
            continue
        elif mode.startswith("remove "):
            user_id = mode[7:].strip()
            user_manager.remove_user(user_id)
            continue
        elif mode == "users":
            user_manager.list_users()
            continue

        user_input = ""
        if mode == "v":
            user_input = get_voice_input()
        elif mode == "t":
            user_input = input(f"🧑 {current_user_name}: ")
        else:
            print("❌ Invalid option. Use 'v', 't', or a command.")
            continue

        user_input = clean_text(user_input)
        if not user_input:
            print("⚠️ Please say or enter something meaningful.")
            continue

        # Get current user's agent executor
        current_user_data = user_manager.get_current_user()
        agent_executor = current_user_data['agent_executor']

        # Use AgentExecutor directly for better conversation handling
        try:
            result = agent_executor.invoke({"input": user_input})
            bot_reply = result.get("output", "I couldn't generate a response.")

            # Increment conversation count
            current_user_data['conversation_count'] += 1

            # Debug: print the result structure
            print("DEBUG result keys:", result.keys())
            print("DEBUG output:", bot_reply)

        except Exception as e:
            print(f"Error: {e}")
            bot_reply = "Sorry, I encountered an error. Please try again."

        # Clean the text - remove special characters, line breaks
        def clean_bot_reply(text):
            # Clean the text
            text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)  # Remove bold
            text = re.sub(r'\*(.*?)\*', r'\1', text)      # Remove italic
            text = re.sub(r'`(.*?)`', r'\1', text)        # Remove code blocks
            text = re.sub(r'\n+', ' ', text)              # Replace newlines with spaces
            text = re.sub(r'\s+', ' ', text)              # Replace multiple spaces with single space
            text = re.sub(r'[^\w\s\.\,\!\?\-\:]', '', text)  # Remove special characters except basic punctuation
            return text.strip()

        # Clean and extract the bot reply
        clean_reply = clean_bot_reply(bot_reply)

        # Print & Speak
        final_reply = truncate_sentences(clean_reply, max_sentences=3)
        print("🤖 Bot:", final_reply)
        speak_output(final_reply)


C:\Users\DELL\AppData\Local\Temp\ipykernel_11396\185684567.py:36: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily = TavilySearchResults()
C:\Users\DELL\AppData\Local\Temp\ipykernel_11396\185684567.py:64: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


🎙 Voice Tutor AI — Multiuser Edition
Commands: 'users', 'switch <user>', 'add <user>', 'remove <user>', 'exit'
Type 'exit' or 'quit' to stop.

New user 'default' added!
👤 Switched to user: default
❌ Invalid option. Use 'v', 't', or a command.
🎤 Speak now (or type)...
 You said: what is quantum mechanics


> Entering new AgentExecutor chain...
Quantum mechanics is a fundamental branch of physics that describes the behavior of matter and energy at the atomic and subatomic scales. It provides a mathematical framework for understanding phenomena that cannot be explained by classical physics, such as:

1. **Wave-Particle Duality**: Particles like electrons and photons exhibit both wave-like and particle-like properties.
2. **Quantization**: Certain properties (e.g., energy levels) are discrete rather than continuous.
3. **Uncertainty Principle**: It is impossible to simultaneously know both the exact position and momentum of a particle (Heisenberg).
4. **Superposition**: Particles can exist